In [ ]:
!pip install wandb -qU
!pip install torchattacks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.5/289.5 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 20.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  

In [ ]:
import numpy as np
import torch
import torchvision
import os
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torchvision.transforms.functional import to_pil_image
from torch.utils.data.dataset import random_split
from torchvision import models
from torch.utils.data import Dataset
from torchattacks import PGD

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("running on the GPU")
else:
    device = torch.device("cpu")
    print("running on the CPU")

running on the GPU


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# 1. Data Preprocessing

In [ ]:
#Hyperparams
batch_size = 64
learning_rate = 1e-3
epochs = 16

In [ ]:
train_transform = transforms.Compose([
        transforms.RandomCrop(32,padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

test_transform =  transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform = train_transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,

                                       download=True, transform = test_transform)

torch.manual_seed(42)
train_len = len(trainset)
num_train = int(train_len * 0.8)
num_val = train_len - num_train
train_set,val_set = random_split(trainset,[num_train, num_val])

trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                              shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                            shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)
print(len(train_set))
print(len(val_set))

100%|██████████| 170498071/170498071 [00:12<00:00, 13241000.84it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
40000
10000


# 2. Models

In [ ]:
#Resnet 50
class Bottleneck(nn.Module):
    def __init__(self, in_channel, mid_channel, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.out_channel = mid_channel * 4
        self.conv1 = nn.Conv2d(in_channel, mid_channel, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(mid_channel)
        self.conv2 = nn.Conv2d(mid_channel, mid_channel, 3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(mid_channel)
        self.conv3 = nn.Conv2d(mid_channel, self.out_channel, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.out_channel)
        self.relu = nn.ReLU(inplace=True)

        if downsample or in_channel != self.out_channel or stride != 1:
            self.downsample_layers = nn.Sequential(
                nn.Conv2d(in_channel, self.out_channel, 1, stride=stride),
                nn.BatchNorm2d(self.out_channel)
            )
        else:
            self.downsample_layers = None

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample_layers is not None:
            identity = self.downsample_layers(identity)

        out += identity
        out = self.relu(out)
        return out




class ResNet50(nn.Module):
  def __init__(self,num_classes=10):
    super(ResNet50,self).__init__()
    self.inplanes = 64
    self.block = Bottleneck
    self.AvgPool = nn.AvgPool2d(4)

    self.conv1 = nn.Conv2d(3,self.inplanes,kernel_size=3,stride=1,padding=1,bias=False)
    self.bn = nn.BatchNorm2d(64)
    self.relu = nn.ReLU(inplace=True)
    ## conv1 통과후->32*32*64->spatial size 그대로!!
    self.layer1 = self._make_layer(64,num_blocks=3,stride=1)

    #layer2 부터는 feature map의 spatial size도 절반씩 감소시켜줄거임!
    self.layer2 = self._make_layer(128,num_blocks=4,stride=2)
    self.layer3 = self._make_layer(256,num_blocks=6,stride=2)
    self.layer4 = self._make_layer(512,num_blocks=3,stride=2)
    #spatial size 작아서 굳이 pooling안해도 될듯?
    self.fc = nn.Linear(512*4,num_classes)

  def _make_layer(self,planes,num_blocks,stride):
    layers = []
    #spatial size를 줄일때: layer의 첫번째 블록만 size 줄이면 됨!
    layers.append(Bottleneck(self.inplanes,planes,stride))
    self.inplanes = planes*4
    for i in range(num_blocks-1):
      layers.append(Bottleneck(self.inplanes,planes))
    return nn.Sequential(*layers)

  def forward(self,x):
     out = self.conv1(x)
     out = self.bn(out)
     out = self.relu(out)
     out = self.layer1(out)
     #print(f'after layer1:{out.shape}')
     out = self.layer2(out)
     #print(f'after layer2:{out.shape}')
     out = self.layer3(out)
     #print(f'after layer3:{out.shape}')
     out = self.layer4(out)
     #print(f'after layer4:{out.shape}')
     out = self.AvgPool(out)
     out = out.view(out.size(0),-1)
     out = self.fc(out)
     #print(out.shape)
     #print("---------end-------------")
     return out


#4. Training Methods

In [ ]:
# Directory to save adversarial examples
import torch.backends.cudnn as cudnn
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = True


def vanilla_train1(dataloader, model, attack, loss_fn, optimizer,iam=None): #train on adv examples only
    model.train()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        if(batch%20==0):
          print(f'batch no. {batch}')
        X = X.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        adv = attack(X,y)
        pred_adv = model(adv) #prediction on adv example
        loss = loss_fn(pred_adv,y)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += (pred_adv.argmax(1) == y).type(torch.float).sum().item()

    train_loss /= num_batches #batch당 평균 train loss->estimate of train loss over entire dataset
    train_acc /= size
    print(f"Training on Adv: \n Accuracy: {(100*train_acc):>0.1f}%, Avg loss: {train_loss:>8f} \n")

    return train_loss, train_acc

def vanilla_train2(dataloader, model, attack, loss_fn, optimizer,iam=None): #adv+benign 1:1
    model.train()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        if(batch%20==0):
          print(f'batch no. {batch}')
        X = X.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        adv = attack(X,y)

        pred = model(X)#prediction on benign example
        pred_adv = model(adv)#prediction on adv example
        loss = torch.mean(loss_fn(pred_adv, y)+loss_fn(pred,y))

        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += (pred_adv.argmax(1) == y).type(torch.float).sum().item()
        train_acc += (pred.argmax(1) == y).type(torch.float).sum().item()

    train_loss /= num_batches
    train_acc /= (2*size)
    print(f"Train on Adv+Benign: \n Accuracy: {(100*train_acc):>0.1f}%, Avg loss: {train_loss:>8f} \n")

    return train_loss, train_acc


def train_alp(dataloader, model, attack, loss_fn, optimizer, lam=0.1): #adv+benign 1:1
    model.train()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        if(batch%20==0):
          print(f'batch no. {batch}')
        X = X.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        adv = attack(X,y)

        pred = model(X)#prediction on benign example
        pred_adv = model(adv)#prediction on adv example
        loss = torch.mean(loss_fn(pred_adv, y)+loss_fn(pred,y))+lam * torch.mean(torch.abs(pred - pred_adv))

        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += (pred_adv.argmax(1) == y).type(torch.float).sum().item()
        train_acc += (pred.argmax(1) == y).type(torch.float).sum().item()

    train_loss /= num_batches
    train_acc /= (2*size)
    print(f"ALP Train: \n Accuracy: {(100*train_acc):>0.1f}%, Avg loss: {train_loss:>8f} \n")

    return train_loss, train_acc

def validation_loop(dataloader, model, attack, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    total_loss,total_acc = 0, 0
    benign_loss, benign_acc = 0, 0
    adv_loss, adv_acc = 0, 0
    total = 0
    for batch,(X,y) in enumerate(dataloader):
      with torch.no_grad():
        X = X.to(device)
        y = y.to(device)
        total += y.size(0)
        pred = model(X)
        benign_loss += loss_fn(pred, y).item()
        benign_acc += (pred.argmax(1) == y).type(torch.float).sum().item()
      X.requires_grad = True
      adv = attack(X,y)
      X.requires_grad = False
      with torch.no_grad():
        pred_adv = model(adv)
        adv_loss += loss_fn(pred_adv, y).item()
        adv_acc += (pred_adv.argmax(1) == y).type(torch.float).sum().item()

    # Compute total loss and accuracy
    benign_loss /= num_batches
    benign_acc /= size
    adv_loss /= num_batches
    adv_acc /= size
    total_loss = (benign_loss + adv_loss) / 2
    total_acc = (benign_acc + adv_acc) / 2
    print('\nTotal benign test accuarcy:', 100.*benign_acc)
    print('Total adversarial test Accuarcy:', 100.*adv_acc)
    print('Total benign test loss:', benign_loss)
    print('Total adversarial test loss:', adv_loss)
    print('**Summary**')
    print('Total validation accuracy:',100*total_acc)
    print('Total validiation loss:',total_loss)

    return total_loss,total_acc,benign_loss,benign_acc,adv_loss,adv_acc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def run(model,optimizer,method ='vanilla_train1',iam=0.1,model_save_name = None):
    # Initialize Weights & Biases
    wandb.init(
        project="First Trials",
        name="experiment_first",
        config={
            "learning_rate": learning_rate,
            "batch_size": batch_size,
            "epochs": epochs,
        })

    # Define device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

   # Move model to device
    model = model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    attack = PGD(model, eps=8/255,alpha=2/255,steps=7)
    train = None
    if(method == 'vanilla_train1'):
      train = vanilla_train1
    elif(method == 'vanilla_train2'):
      train = vanilla_train2
    else:
      train = train_alp
    best_validation_acc = 0.0
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loss, train_acc = train(trainloader, model, attack,loss_fn, optimizer,iam)
        total_loss,total_acc,benign_loss,benign_acc,adv_loss,adv_acc = validation_loop(valloader, model, attack, loss_fn)

        # sample_batch, _ = next(iter(trainloader))
        # sample_img = to_pil_image(sample_batch[0])
        # sample_img = wandb.Image(sample_img, caption="sample image")

        # images = [to_pil_image(image) for image in sample_batch[:5]]  # multiple images

        wandb.log({
            "train_acc": train_acc,
            "train_loss": train_loss,
            "benign_loss":benign_loss,
            "benign_acc":benign_acc,
            "adv_loss":adv_loss,
            "adv_acc":adv_acc,
            "total_acc":total_acc,
            "total_loss":total_loss,
            # "example": sample_img,
            # "examples": [wandb.Image(image) for image in images],
        })

        if total_acc > best_validation_acc:
            best_validation_acc = total_acc
            print(f"Model at epoch{t} saved")
            path = F"/content/drive/MyDrive/AI_project/SaveModel/best_{model_save_name}"
            torch.save(model.state_dict(), path)
        path = F"/content/drive/MyDrive/AI_project/SaveModel/last_{model_save_name}"
        torch.save(model.state_dict(), path)

    print("Done!")
    wandb.finish()

In [ ]:
#vanilla model2
torch.manual_seed(42)
vanilla_model2 = ResNet50().to(device)
optimizer = torch.optim.SGD(vanilla_model2.parameters(), lr=learning_rate, momentum=0.9)
#optimizer = torch.optim.Adam(vanilla_model2.parameters(), lr=learning_rate)
run(vanilla_model2,optimizer,'vanilla_train2',model_save_name = 'vanilla2.pt')

wandb: Currently logged in as: dayoonl055 (dayoonlee). Use `wandb login --relogin` to force relogin


Epoch 1
-------------------------------
batch no. 0
batch no. 20
batch no. 40
batch no. 60
batch no. 80
batch no. 100
batch no. 120
batch no. 140
batch no. 160
batch no. 180
batch no. 200
batch no. 220
batch no. 240
batch no. 260
batch no. 280
batch no. 300
batch no. 320
batch no. 340
batch no. 360
batch no. 380
batch no. 400
batch no. 420
batch no. 440
batch no. 460
batch no. 480
batch no. 500
batch no. 520
batch no. 540
batch no. 560
batch no. 580
batch no. 600
batch no. 620
Train on Adv+Benign: 
 Accuracy: 31.8%, Avg loss: 3.786319 


Total benign test accuarcy: 39.33
Total adversarial test Accuarcy: 9.19
Total benign test loss: 1.6504408705766034
Total adversarial test loss: 3.2044336461717156
**Summary**
Total validation accuracy: 24.259999999999998
Total validiation loss: 2.4274372583741597
Model at epoch0 saved
Epoch 2
-------------------------------
batch no. 0
batch no. 20
batch no. 40
batch no. 60
batch no. 80
batch no. 100
batch no. 120
batch no. 140
batch no. 160
batch no. 

adv_acc,▂▁▄▁▃▅▇▃▆▇▅▅██▇█
adv_loss,▁▅▅▆▄▂▆▆▇▆▇█▄▃█▅
benign_acc,▁▂▃▃▃▅▆▆▆▄▆▇▇▇▆█
benign_loss,█▇▆▆▇▅▃▃▃▇▃▂▄▂▄▁
total_acc,▁▂▃▃▃▄▆▆▆▅▆▇▇▇▇█
total_loss,▃▇▆▇▆▂▅▅▆█▆▇▄▁█▃
train_acc,▁▃▄▅▅▆▆▇▇▇▇▇████
train_loss,█▆▅▄▄▃▃▂▂▂▂▂▁▁▁▁
adv_acc,0.1547
adv_loss,3.80436
benign_acc,0.6181


In [ ]:
#alp model
torch.manual_seed(42)
alp_model = ResNet50().to(device)
optimizer = torch.optim.SGD(alp_model.parameters(), lr=learning_rate, momentum=0.9)
run(alp_model,optimizer,'alp_train',iam=0.1,model_save_name = 'alp.pt')

Epoch 1
-------------------------------
batch no. 0
batch no. 20
batch no. 40
batch no. 60
batch no. 80
batch no. 100
batch no. 120
batch no. 140
batch no. 160
batch no. 180
batch no. 200
batch no. 220
batch no. 240
batch no. 260
batch no. 280
batch no. 300
batch no. 320
batch no. 340
batch no. 360
batch no. 380
batch no. 400
batch no. 420
batch no. 440
batch no. 460
batch no. 480
batch no. 500
batch no. 520
batch no. 540
batch no. 560
batch no. 580
batch no. 600
batch no. 620
ALP Train: 
 Accuracy: 31.7%, Avg loss: 3.823396 


Total benign test accuarcy: 40.589999999999996
Total adversarial test Accuarcy: 10.95
Total benign test loss: 1.638281203379297
Total adversarial test loss: 3.059768385188595
**Summary**
Total validation accuracy: 25.77
Total validiation loss: 2.349024794283946
Model at epoch0 saved
Epoch 2
-------------------------------
batch no. 0
batch no. 20
batch no. 40
batch no. 60
batch no. 80
batch no. 100
batch no. 120
batch no. 140
batch no. 160
batch no. 180
batch no

adv_acc,▃▁▃▁▃▄▅▅▆▆▇▇█▇██
adv_loss,▁▅▅█▅▅▃▆▆▆▄▅▅▄▇▅
benign_acc,▁▂▄▅▅▅▆▇▆▆▇██▇██
benign_loss,█▇▅▄▄▅▃▂▂▃▁▁▁▂▁▁
total_acc,▁▁▃▄▄▄▅▆▆▆▇▇█▇██
total_loss,▃█▆█▅▅▂▄▅▆▁▂▂▂▄▂
train_acc,▁▃▄▅▅▆▆▇▇▇▇▇████
train_loss,█▆▅▄▄▃▃▂▂▂▂▂▁▁▁▁
adv_acc,0.1577
adv_loss,3.54467
benign_acc,0.6558
